In [1]:
# Downloads:
# !wget nlp.stanford.edu/data/glove.6B.zip

--2021-01-24 20:11:54--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-01-24 20:11:55--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-01-24 20:11:56--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [2]:
# IMPORTS: Standard Imports
import pandas as pd
import numpy as np
import io
import zipfile
import re

In [3]:
'''from google.colab import drive
drive.mount('/content/drive')'''

Mounted at /content/drive


In [3]:
# Unzipping Downloaded file
'''with zipfile.ZipFile('glove.6B.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/drive/My Drive/Makeathon/GloVe')'''

In [5]:
!ls drive/MyDrive/Makeathon/GloVe

glove.6B.100d.txt  glove.6B.200d.txt  glove.6B.300d.txt  glove.6B.50d.txt


In [4]:
def loadGloveModel(File):
    """
    Loads the GloVe Model from a particular .txt file.
    """
    print("Loading Glove Model")
    f = open(File,'r')
    gloveModel = {}
    for line in f:
        splitLines = line.split()
        word = splitLines[0]
        wordEmbedding = np.array([float(value) for value in splitLines[1:]])
        gloveModel[word] = wordEmbedding
    print(len(gloveModel)," words loaded!")
    return gloveModel

In [6]:
# /content/drive/My Drive/Makeathon/
gloveModel = loadGloveModel('/content/drive/My Drive/Makeathon/GloVe/glove.6B.50d.txt')
print(gloveModel['hello'])

Loading Glove Model
400000  words loaded!
[-0.38497   0.80092   0.064106 -0.28355  -0.026759 -0.34532  -0.64253
 -0.11729  -0.33257   0.55243  -0.087813  0.9035    0.47102   0.56657
  0.6985   -0.35229  -0.86542   0.90573   0.03576  -0.071705 -0.12327
  0.54923   0.47005   0.35572   1.2611   -0.67581  -0.94983   0.68666
  0.3871   -1.3492    0.63512   0.46416  -0.48814   0.83827  -0.9246
 -0.33722   0.53741  -1.0616   -0.081403 -0.67111   0.30923  -0.3923
 -0.55002  -0.68827   0.58049  -0.11626   0.013139 -0.57654   0.048833
  0.67204 ]


In [8]:
# Loading train and test datasets. test-data is Evaluation.csv, just renamed
train_dataframe = pd.read_csv("word-meaning-examples.csv", sep = ',')
test_dataframe = pd.read_csv("test-data.csv")

In [9]:
train_dataframe = train_dataframe.loc[:, :'Meaning']

In [10]:
train_dict = train_dataframe.set_index('Word').T.to_dict('list')
test_dict = test_dataframe.set_index('Masked').T.to_dict('list')

<ipython-input-10-192a8984c094>:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  train_dict = train_dataframe.set_index('Word').T.to_dict('list')


'\n  Creates a vector representation of sentences. \n  Uses a dictionary of words and their meanings as input,\n  converts each word of the sentence into a word vector \n  representation, and averages them to form a single\n  vector representation for each word.\n  \n  '

In [26]:
def create_sentence_vectors(dictionary):
  
  """
  Creates a vector representation of sentences. 
  Uses a dictionary of words and their meanings as input,
  converts each word of the sentence into a word vector 
  representation, and averages them to form a single
  vector representation for each word.
  
  """
  # Preprocessing
  for word in dictionary.keys():
    dictionary[word] = dictionary[word][0]
  dict_vec = {}
  preprocessed_meaning = []
  for word in dictionary.keys():
    sentence_vector = []
    # print("Word:", word)
    meaning = dictionary[word]
    # Checking whether the meaning is in string form or not.
    if isinstance(meaning, str) is False:
      pass
    else:
      # Removing special characters, and dividing each word in the sentence to seperate lists
      preprocessed_meaning = " ".join(re.findall(r"[a-zA-Z0-9]+", meaning)).split()
      # Iterating through each word and finding corresponding GloVe vector
      for i in preprocessed_meaning:
        # Checking if word is present in the list of defined words
        if i in gloveModel.keys():
          wordvector = gloveModel[i]
        else:
          pass  
        sentence_vector.append(wordvector)
      dict_vec[word] = sentence_vector
  # Conversion into one vector
  meanings_vectorized = {}
  for word in dict_vec.keys():
    total_vector = np.zeros(50,)
    index = 0
    for vector in dict_vec[word]:
        total_vector += vector
        index += 1

    total_vector /= index 
    meanings_vectorized[word] = total_vector
  return meanings_vectorized


In [13]:
train_dict_vectors = create_sentence_vectors(train_dict)
test_dict_vectors = create_sentence_vectors(test_dict)

In [14]:
test_dict_vectors['C o _ _ n t h']

array([ 0.54275953,  0.32810435, -0.24265694,  0.11212776,  0.26464259,
       -0.07517229, -0.59884588, -0.13959435, -0.15745542, -0.22009911,
        0.11662318, -0.20112004, -0.09344824, -0.32191929,  0.07800515,
        0.21148671,  0.04706547,  0.13835541, -0.75366465,  0.08472859,
        0.16989406, -0.09572382, -0.27618641, -0.01999353,  0.12779588,
       -1.74803529, -0.48220176,  0.04652724,  0.03563059, -0.27824765,
        3.40601765, -0.27075859, -0.23650412, -0.43757535,  0.23031096,
        0.07186159,  0.10967053,  0.14501253, -0.02941979,  0.27797647,
       -0.11065612,  0.022959  ,  0.12191429, -0.05813229, -0.26083382,
        0.33865094, -0.16533843, -0.25876953,  0.04781965, -0.59547699])

In [22]:
similar_words = {}
for masked_word in test_dict_vectors:
  word_list = []
  for vocabulary_word in train_dict_vectors:  
    distance = 0.0
    u = test_dict_vectors[masked_word]
    v = train_dict_vectors[vocabulary_word]    
    # Compute the dot product between u and v
    dot = np.dot(u, v)
    # Compute the L2 norm of u
    norm_u = np.sqrt(np.sum(u**2))
    # Compute the L2 norm of v
    norm_v = np.sqrt(np.sum(v**2))

    # Compute the cosine similarity
    cosine_similarity = dot/(norm_u * norm_v)
    if (cosine_similarity > 0.9):
      # print(f"Appending {vocabulary_word} to word list")
      if len(word_list) > 3:
            pass
      else:
            word_list.append(vocabulary_word)
  similar_words[masked_word] = word_list
  


In [23]:
similar_words

{'C o _ _ n t h': ['Abattoir', 'Abnormal', 'Aboard', 'Abode '],
 '_ e c e i _ e': ['A bed of roses', 'A Priori', 'Abandon', 'Abase'],
 '_ o l l a g _': ['A bed of roses', 'A Priori', 'Abduct', 'Abject'],
 't _ _ s h': ['A bed of roses', 'A Priori', 'Abase', 'Abdicate'],
 'D e _ a w _ r _': ['Abdominal', 'Aberrant', 'Abnormal', 'Aboard'],
 'p a l e _ t _': ['Abbess', 'Abnormal', 'Abnormality', 'Aboriginal'],
 'i c _ _ a _ e _ r o n': [],
 'B a _ _ a c': ['Hasidic', 'Rococo'],
 '_ a l a _ s i _': ['Abnormal', 'Abode ', 'Abolish', 'Abolition'],
 's _ _ i _ o s i s': ['Cholesterol', 'Cyanosis', 'Hypertonic', 'Hypotrophy'],
 's _ b _ _ _ s i b l e': ['A bed of roses', 'A Priori', 'Abduct', 'Abet'],
 'f _ o t p _ d': ['Adolescence', 'Bayonet', 'Bonnet', 'Casualty'],
 'l a n g _ _': ['Buzzard', 'Cilia', 'Claw', 'Hare'],
 '_ _ s c a n': ['Abbess', 'Abnormal', 'Abode ', 'Aboriginal'],
 'u n i _ e _ s _': ['A bed of roses', 'A Priori', 'Abdicate', 'Aberration'],
 'e x _ _ s e': ['A bed of roses'